In [1]:
# Call the relevant python packages
import numpy as np
import pandas as pd
import glob, os                          # Operating system module and pathnames

import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.io import ascii, fits
from astropy.table import join, hstack, Table, Column, vstack
from astropy.coordinates import Angle, Latitude, Longitude, SkyCoord

#import SED Fitter module
from sedfitter import fit
from sedfitter import plot
from sedfitter.extinction import Extinction
from sedfitter import plot_params_1d as param1d
from sedfitter import filter_output

# Set colour palette & plot layouts
import seaborn as sns     #; sns.set()
sns.set_context("paper")   # "paper", "notebook", "talk", "poster"

# Output plots in Notebook
#%matplotlib notebook
%matplotlib inline

In [2]:
# Read in the catalogue
file = '/scratch/s1625982/NGC6822/env/NGC6822_filtered.csv'
data = ascii.read(file, data_start=1)

# Print the number of YSOs found in the imported table
YSO_num=len(data)
print("The number of YSOs in the table is",YSO_num)

# Set up new columns for the flux
data['J flux']=0.
data['H flux']=0.
data['K flux']=0.
data['36 flux']=0.
data['45 flux']=0.
data['58 flux']=0.
data['80 flux']=0.
data['24 flux']=0.

# Set up new columns for the flux errors
# By default set these to 0.2
data['J flux err']=0.2
data['H flux err']=0.2
data['K flux err']=0.2
data['36 flux err']=0.2
data['45 flux err']=0.2
data['58 flux err']=0.2
data['80 flux err']=0.2
data['24 flux err']=0.2

# Set up new columns for the integer assigned to each source (0-4).
# NB: this program assumes sources are either valid or upper limits.
# Hence, flags are either 1 (valid) or 3 (upper limit).
# For SPs, flag all 8 and 24 micron data as upper limits as confusion is highly likely in the SEDs
data['J int']=1
data['H int']=1
data['K int']=1
data['36 int']=1
data['45 int']=1
data['58 int']=1
data['80 int']=3
data['24 int']=3

# Assign the integer describing source reliability
def write_column(idx,name,flag):
    for i in range(len(idx[0])):
        #if source is upper limit write 3 flag
        #if source is bad photometry write 0 flag
        data[name][idx[0][i]]=flag

# Set up a new column for sources having 2JHK, 36, 45 but no 80 or 24
data['confused?']=0
idx_confused=np.where((data['HJKNum']>=2)&(data['36 data?']==1)&(data['45 data?']==1)&(data['24 data?']==0)&(data['80 data?']==0))
write_column(idx_confused,'confused?',1)
print("Th number of sources with 2 of JHK, 36 and 45 but no 80 or 24 micron flux is",len(idx_confused[0]))

err=0.2
# Create indices to find which sources are upper limits
idx_UL_J=np.where(((data['eJmag']<=0)|(data['eJmag']>err))&(data['Jmag']>0))
write_column(idx_UL_J,'J int',3)
idx_UL_H=np.where(((data['eHmag']<=0)|(data['eHmag']>err))&(data['Hmag']>0))
write_column(idx_UL_H,'H int',3)
idx_UL_K=np.where(((data['eKmag']<=0)|(data['eKmag']>err))&(data['Kmag']>0))
write_column(idx_UL_K,'K int',3)
idx_UL_36=np.where(((data['e36mag']<=0)|(data['e36mag']>err))&(data['k36mag']>0))
write_column(idx_UL_36,'36 int',3)
idx_UL_45=np.where(((data['e45mag']<=0)|(data['e45mag']>err))&(data['k45mag']>0))
write_column(idx_UL_45,'45 int',3)
idx_UL_58=np.where(((data['e58mag']<=0)|(data['e58mag']>err))&(data['k58mag']>0))
write_column(idx_UL_58,'58 int',3)
idx_UL_80=np.where(((data['e80mag']<=0)|(data['e80mag']>err))&(data['k80mag']>0))
write_column(idx_UL_80,'80 int',3)
idx_UL_24=np.where(((data['e24mag']<=0)|(data['e24mag']>err))&(data['k24mag']>0))
write_column(idx_UL_24,'24 int',3)

# Create indices to find which sources are bad photometry
idx_BP_J=np.where(((data['eJmag']<=0)|(data['eJmag']>err))&(data['Jmag']<=0))
write_column(idx_BP_J,'J int',0)
idx_BP_H=np.where(((data['eHmag']<=0)|(data['eHmag']>err))&(data['Hmag']<=0))
write_column(idx_BP_H,'H int',0)
idx_BP_K=np.where(((data['eKmag']<=0)|(data['eKmag']>err))&(data['Kmag']<=0))
write_column(idx_BP_K,'K int',0)
idx_BP_36=np.where(((data['e36mag']<=0)|(data['e36mag']>err))&(data['k36mag']<=0))
write_column(idx_BP_36,'36 int',0)
idx_BP_45=np.where(((data['e45mag']<=0)|(data['e45mag']>err))&(data['k45mag']<=0))
write_column(idx_BP_45,'45 int',0)
idx_BP_58=np.where(((data['e58mag']<=0)|(data['e58mag']>err))&(data['k58mag']<=0))
write_column(idx_BP_58,'58 int',0)
idx_BP_80=np.where(((data['e80mag']<=0)|(data['e80mag']>err))&(data['k80mag']<=0))
write_column(idx_BP_80,'80 int',0)
idx_BP_24=np.where(((data['e24mag']<=0)|(data['e24mag']>err))&(data['k24mag']<=0))
write_column(idx_BP_24,'24 int',0)

# Hardcode zero points of filters for Vega system.
# Fluxes and their errors are given in mJy
F_0_J, F_0_err_J, m_0_J = 1601000, 0.0, -0.016
F_0_H, F_0_err_H, m_0_H = 1034000, 0.0, 0.018
F_0_K, F_0_err_K, m_0_K = 670300, 0.0, 0.008
F_0_36, F_0_err_36, m_0_36 = 280900, 4100, 0.0
F_0_45, F_0_err_45, m_0_45 = 179700, 2600, 0.0
F_0_58, F_0_err_58,m_0_58 = 115000, 1700, 0.0
F_0_80, F_0_err_80, m_0_80 = 1034000, 900, 0.0
F_0_24, F_0_err_24, m_0_24 = 64900, 110, 0.0

# Define lists for Fv, Fv_0, m_0 etc.
int_list = ['J int','H int','K int','36 int','45 int','58 int','80 int','24 int']
flux_list = ['J flux','H flux','K flux','36 flux','45 flux','58 flux','80 flux','24 flux']
flux_err_list = ['J flux err','H flux err','K flux err','36 flux err','45 flux err','58 flux err','80 flux err','24 flux err']
F_0_list = [F_0_J,F_0_H,F_0_K,F_0_36,F_0_45,F_0_58,F_0_80,F_0_24]
F_0_err_list = [F_0_err_J,F_0_err_H,F_0_err_K,F_0_err_36,F_0_err_45,F_0_err_58,F_0_err_80,F_0_err_24]
m_0_list = [m_0_J,m_0_H,m_0_K,m_0_36,m_0_45,m_0_58,m_0_80,m_0_24]
mag_list = ['Jmag','Hmag','Kmag','k36mag','k45mag','k58mag','k80mag','k24mag']
mag_err_list = ['eJmag','eHmag','eKmag','e36mag','e45mag','e58mag','e80mag','e24mag']

# Write flux and flux error calculations out for each source in each filter
for i in range(len(flux_list)):
        data[flux_list[i]]=F_0_list[i]*np.power(10,0.4*(m_0_list[i]-data[mag_list[i]]))
        #data[flux_err_list[i]]=data[flux_list[i]]*np.sqrt((np.square(0.921*data[mag_err_list[i]]))-np.square(F_0_err_list[i]/F_0_list[i]))
        for j in range(YSO_num):
            if data[int_list[i]][j]==3:
                data[flux_err_list[i]][j]=1.0
            else:
                data[flux_err_list[i]][j]=0.1*data[flux_list[i]][j]

The number of YSOs in the table is 518
Th number of sources with 2 of JHK, 36 and 45 but no 80 or 24 micron flux is 0


In [4]:
data.show_in_notebook()

ID,RA,Dec,k36mag,e36mag,d36mag,k45mag,e45mag,d45mag,k58mag,e58mag,d58mag,k80mag,e80mag,d80mag,k24mag,e24mag,dstArcSec,RAJ2000,DEJ2000,Jmag,eJmag,Hmag,eHmag,Kmag,eKmag,c1a,c1b,c1c,c2a,c2b,c2c,c3a,c3b,c3c,c4a,c4b,c4c,c5a,c5b,c5c,c6a,c6b,c6c,c7,c8,c9a,c9b,c9c,c11b,c11c,c1,c2,c3,c4,c5,c6,c9,c10,c11,24 data?,36 data?,45 data?,58 data?,80 data?,SpNum,H data?,J data?,K data?,HJKNum,c2 met?,c3 met?,c4 met?,c5 met?,c6 met?,c9 met?,c11 met?,sp rel?,HJK rel?,YSO Score,24 without 58/80,New name,Total bands,J flux,H flux,K flux,36 flux,45 flux,58 flux,80 flux,24 flux,J flux err,H flux err,K flux err,36 flux err,45 flux err,58 flux err,80 flux err,24 flux err,J int,H int,K int,36 int,45 int,58 int,80 int,24 int,confused?
3776,296.02927,-14.68646,17.16,0.05,0.12,16.14,0.05,-0.04,15.31,0.08,0.48,14.02,0.07,0.06,10.02,0.04,0.715527028018423,296.029144,-14.686303,19.31,0.164,18.867,0.131,18.045,0.089,1,1,1,0,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,3,1,1,1,1,1,1,0,0,1,1,1,1,1,5,1,1,1,3,1,1,1,1,1,1,0,1,1,6,0,1,8,0.029784569818326796,0.02984836444558514,0.04087607801783847,0.03841950271098733,0.06288514651499574,0.08643663281209352,2.549884674700303,6.371544150199699,0.00297845698183268,0.0029848364445585144,0.004087607801783847,0.0038419502710987335,0.006288514651499575,0.008643663281209352,1.0,1.0,1,1,1,1,1,1,3,3,0
8145,296.04298,-14.80325,17.14,0.11,1.04,17.04,0.16,1.26,15.96,0.15,0.83,13.55,0.08,1.15,9.05,0.03,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,1,1,1,1,1,5,0,0,0,0,0,0,1,1,1,1,0,1,0,4,0,2,5,inf,inf,inf,0.039133774594733495,0.027450362066538177,0.047500462728868514,3.9311583579545952,15.568425644436473,inf,inf,inf,0.00391337745947335,0.002745036206653818,0.004750046272886852,1.0,1.0,0,0,0,1,1,1,3,3,0
2235,296.04335,-14.80379,15.89,0.08,0.83,15.55,0.09,0.83,15.38,0.12,1.23,12.4,0.04,0.61,8.95,0.03,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1,0,0,1,1,1,1,1,5,0,0,0,0,0,0,1,1,1,1,0,1,0,4,0,3,5,inf,inf,inf,0.1237518611589906,0.10827995761756193,0.0810397029312217,11.33758454812051,17.070439267401024,inf,inf,inf,0.012375186115899061,0.010827995761756193,0.008103970293122171,1.0,1.0,0,0,0,1,1,1,3,3,0
19111,296.04821,-14.68252,18.83,0.1,-0.32,18.01,0.09,0.1,16.28,0.18,-9999.0,14.97,0.08,-0.93,11.27,0.08,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,1,1,1,1,1,5,0,0,0,0,1,0,1,0,0,1,0,1,0,3,0,5,5,inf,inf,inf,0.008251857872359939,0.011234353289179056,0.0353751133695181,1.0629688522627727,2.014859172695303,inf,inf,inf,0.0008251857872359939,0.0011234353289179056,0.0035375113369518103,1.0,1.0,0,0,0,1,1,1,3,3,0
3480,296.05655,-14.87977,17.01,0.15,0.79,16.05,0.09,0.06,15.69,0.12,-0.1,15.36,0.13,0.1,9.88,0.04,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,1,1,1,5,0,0,0,0,1,0,0,1,1,0,0,1,0,3,0,7,5,inf,inf,inf,0.044111491173729374,0.06832003451880471,0.060911296047540676,0.7421992971745878,7.248442478095836,inf,inf,inf,0.004411149117372937,0.006832003451880471,0.006091129604754068,1.0,1.0,0,0,0,1,1,1,3,3,0
2491,296.05716,-14.83338,16.11,0.11,0.25,15.68,0.12,0.3,15.89,0.09,0.66,13.55,0.05,0.21,9.92,0.04,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0,0,1,1,1,1,1,5,0,0,0,0,0,0,1,1,1,1,0,1,0,4,0,8,5,inf,inf,inf,0.10105358824524803,0.09606121538363575,0.05066380930325354,3.9311583579545952,6.986259236457629,inf,inf,inf,0.010105358824524804,0.009606121538363576,0.005066380930325354,1.0,1.0,0,0,0,1,1,1,3,3,0
28211,296.05902,-14.7746,18.77,0.16,0.01,18.82,0.17,-0.08,16.3,0.12,-9999.0,14.82,0.09,-0.81,11.3,0.07,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,0,0,0,0,0,1,0,0,1,1,0

In [3]:
# Write out table in the right format for SED fitter
ascii.write(data[['New name','RA','Dec','J int','H int','K int','36 int','45 int','58 int','80 int','24 int',
                  'J flux','J flux err','H flux','H flux err','K flux','K flux err','36 flux','36 flux err',
                  '45 flux','45 flux err','58 flux','58 flux err','80 flux','80 flux err','24 flux','24 flux err']],
            'NGC6822_YSOs_SED_format_SPs.txt', format='no_header', fast_writer=False)

# NB: THE USER IS REQUIRED TO FIND AND REPLACE 'INF' IN THE FILE WITH WHATEVER THEY WISH TO USE
#     TO DENOTE UPPER LIMITS (E.G. -9999)

In [6]:
# Define path to models
model_dir = '/scratch/s1625982/NGC6822/env/models_kurucz/'

# Read in extinction law)
extinction = Extinction.from_file('/scratch/s1625982/NGC6822/env/Input_SED/kmh94.par',
                                  columns=[0, 3], wav_unit=u.micron, chi_unit=u.cm**2 / u.g)

# Define filters and apertures
filters = ['2J', '2H', '2K', 'I1', 'I2', 'I3', 'I4', 'M1']
apertures = [2., 2., 2., 1.2, 1.2, 1.2, 1.2, 6.] * u.arcsec


# Run the fitting
# NB: THE SP MODELS RUN MUCH FASTER THAN THE YSOS (~500 SOURCES IN 2 SECONDS)

SED_input_file = '/scratch/s1625982/NGC6822/env/NGC6822_YSOs_SED_format_SPs.txt'
SED_output_file = '/scratch/s1625982/NGC6822/env/Output_SED/SP/sp_fit_output.fitinfo'
data = ascii.read(SED_input_file, data_start=1)
fit(SED_input_file, filters, apertures, model_dir, SED_output_file,
    extinction_law=extinction,
    distance_range=[450., 530.] * u.kpc,     # Distance range here is for NGC 6822
    av_range=[0., 40.])                      # A_V range here is for NGC 6822

 ------------------------------------------------------------
  => Model parameters
 ------------------------------------------------------------

   Models              :  Kurucz stellar photospheres
   Log[d] stepping     :  0.02

 ------------------------------------------------------------
  => Reading in convolved fluxes
 ------------------------------------------------------------

   Reading /scratch/s1625982/NGC6822/env/models_kurucz//convolved/2J.fits
   Reading /scratch/s1625982/NGC6822/env/models_kurucz//convolved/2H.fits
   Reading /scratch/s1625982/NGC6822/env/models_kurucz//convolved/2K.fits
   Reading /scratch/s1625982/NGC6822/env/models_kurucz//convolved/I1.fits
   Reading /scratch/s1625982/NGC6822/env/models_kurucz//convolved/I2.fits
   Reading /scratch/s1625982/NGC6822/env/models_kurucz//convolved/I3.fits
   Reading /scratch/s1625982/NGC6822/env/models_kurucz//convolved/I4.fits
   Reading /scratch/s1625982/NGC6822/env/models_kurucz//convolved/M1.fits
 ----------------

In [4]:
# Filter an output file by the value of Xbest^2.
# Those with a X^2 greater than the second parameter will be sorted to _good file
filter_output('/scratch/s1625982/NGC6822/env/Output_SED/SP/sp_fit_output.fitinfo', chi=25.)

# Plot the fits for the output file
from sedfitter import plot
SED_good_output_file = '/scratch/s1625982/NGC6822/env/Output_SED/SP/sp_fit_output.fitinfo_good'
SED_bad_output_file = '/scratch/s1625982/NGC6822/env/Output_SED/SP/sp_fit_output.fitinfo_bad'

plot(SED_good_output_file, '/scratch/s1625982/NGC6822/env/Output_SED/SP/sp_SEDs_good')
plot(SED_bad_output_file, '/scratch/s1625982/NGC6822/env/Output_SED/SP/sp_SEDs_bad')

Delete directory /scratch/s1625982/NGC6822/env/Output_SED/SP/sp1to518good? [y/[n]] y



In [ ]:
# Plot a graph of parameters
parameter1 = 'RSTAR'
parameter2 = 'TSTAR'
path1 = '2006_YSO_'+parameter1+'1to100'
path2 = '2006_YSO_'+parameter1+'_and_'+parameter2+'1to100'

#plot_params_1d('/scratch/s1625982/NGC6822/env/Output_SED/1to100.fitinfo', parameter1, path1)
plot_params_2d('/scratch/s1625982/NGC6822/env/Output_SED/1to100.fitinfo', parameter1, parameter2, path2)